# Direct Marketing with Amazon SageMaker XGBoost

Last update: February 5th, 2020

In [ ]:
%%sh
pip -q install --upgrade pip
pip -q install sagemaker awscli boto3 smdebug --upgrade

In [ ]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

___
# PART 1 - Downloading and processing the dataset

Let's start by downloading the [direct marketing dataset](https://archive.ics.uci.edu/ml/datasets/bank+marketing) from UCI's ML Repository.

In [ ]:
!wget -N https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
!unzip -o bank-additional.zip

In [ ]:
!head ./bank-additional/bank-additional-full.csv

We need to load this CSV file, inspect it, pre-process it, etc. Please don't write custom Python code to do this!

Instead, developers typically use libraries such as:
* Pandas: a library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language: https://pandas.pydata.org/.
* Numpy: a fundamental package for scientific computing with Python: http://www.numpy.org/

Along the way, we'll use functions from these two libraries. You should definitely become familiar with them, they will make your life much easier when working with large datasets.

In [ ]:
import numpy as np  # For matrix operations and numerical processing
import pandas as pd # For munging tabular data

Let's read the CSV file into a Pandas data frame and take a look at the first few lines.

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html
data = pd.read_csv('./bank-additional/bank-additional-full.csv', sep=';')
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 50)         # Keep the output on one page
data[:10] # Show the first 10 lines

In [ ]:
data.shape # (number of lines, number of columns)

The two classes are extremely unbalanced and it could be a problem for our classifier.

In [ ]:
one_class = data[data['y']=='yes']
one_class_count = one_class.shape[0]
print("Positive samples: %d" % one_class_count)

zero_class = data[data['y']=='no']
zero_class_count = zero_class.shape[0]
print("Negative samples: %d" % zero_class_count)

zero_to_one_ratio = zero_class_count/one_class_count
print("Ratio: %.2f" % zero_to_one_ratio)

### Transforming the dataset
Cleaning up data is part of nearly every machine learning project.  It arguably presents the biggest risk if done incorrectly and is one of the more subjective aspects in the process.

First of all, many records have the value of "999" for pdays, number of days that passed by after a client was last contacted. It is very likely to be a magic number to represent that no contact was made before. Considering that, we create a new column called "no_previous_contact", then grant it value of "1" when pdays is 999 and "0" otherwise.

In [ ]:
[np.min(data['pdays']), np.max(data['pdays'])]

In [ ]:
# Indicator variable to capture when pdays takes a value of 999
# https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.where.html
data['no_previous_contact'] = np.where(data['pdays'] == 999, 1, 0)
data = data.drop(['pdays'], axis=1)

In the "job" column, there are categories that mean the customer is not working, e.g., "student", "retire", and "unemployed". Since it is very likely whether or not a customer is working will affect his/her decision to enroll in the term deposit, we generate a new column to show whether the customer is working based on "job" column.

In [ ]:
data['job'].value_counts()

In [ ]:
# Indicator for individuals not actively employed
# https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.in1d.html
data['not_working'] = np.where(np.in1d(data['job'], ['student', 'retired', 'unemployed']), 1, 0)

Last but not the least, we convert categorical to numeric, as is suggested above.

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html
model_data = pd.get_dummies(data)  # Convert categorical variables to sets of indicators
model_data[:10]

As you can see, each categorical column (job, marital, education, etc.) has been replaced by a set of new columns, one for each possible value in the category. Accordingly, we now have 67 columns instead of 21.

In [ ]:
model_data.shape

### Splitting the dataset

We'll then split the dataset into training (70%), validation (20%), and test (10%) datasets and convert the datasets to the right format the algorithm expects. We will use training and validation datasets during training and we will try to maximize the accuracy on the validation dataset.
 
Once the model has been deployed, we'll use the test dataset to evaluate its performance.

Amazon SageMaker's XGBoost algorithm expects data in the libSVM or CSV data format.  For this example, we'll stick to CSV.  Note that the first column must be the target variable and the CSV should not include headers.  Also, notice that although repetitive it's easiest to do this after the train|validation|test split rather than before.  This avoids any misalignment issues due to random reordering.

In [ ]:
# Set the seed to 123 for reproductibility
# https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.DataFrame.sample.html
# https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.split.html
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=123), 
                                                  [int(0.7 * len(model_data)), int(0.9*len(model_data))])  

# Drop the two columns for 'yes' and 'no' and add 'yes' back as first column of the dataframe
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
pd.concat([validation_data['y_yes'], validation_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('validation.csv', index=False, header=False)
#pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)

# Dropping the target value, as we will use this CSV file for batch transform
test_data.drop(['y_no', 'y_yes'], axis=1).to_csv('test.csv', index=False, header=False)

In [ ]:
!ls -l *.csv

Now we'll copy the files to S3 for Amazon SageMaker training to pickup.

In [ ]:
import sagemaker
import boto3, os

print (sagemaker.__version__)

bucket = sagemaker.Session().default_bucket()                     
prefix = 'sagemaker/DEMO-xgboost-dm'

boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')

SageMaker needs to know where the training and validation sets are located, so let's define that.

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')
s3_data = {'train': s3_input_train, 'validation': s3_input_validation}

---
# PART 2 - Training our model

The problem we're trying to solve is a classification problem: will a given customer react positively to our marketing offer or not? In order to answer this question, let's train a classification model with XGBoost, a popular open source project available in SageMaker.

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri

from sagemaker.estimator import Estimator
# https://sagemaker.readthedocs.io/en/stable/estimators.html

from sagemaker.debugger import rule_configs, Rule, DebuggerHookConfig, CollectionConfig
# https://sagemaker.readthedocs.io/en/stable/debugger.html
# https://docs.aws.amazon.com/sagemaker/latest/dg/class-imbalance.html

sess = sagemaker.Session()
region = boto3.Session().region_name    

container = get_image_uri(region, 'xgboost', repo_version='0.90-2')

save_interval = '1'

xgb = Estimator(
    
    container,                                               # The algorithm (XGBoost)
    role=sagemaker.get_execution_role(),                     # IAM permissions for SageMaker
    sagemaker_session=sess,                                  # Technical object
                                    
    input_mode='File',                                       # Copy the dataset and then train
    output_path='s3://{}/{}/output'.format(bucket, prefix),  # Save the model here
                                    
    train_instance_count=1,                                  # Instance requirements
    train_instance_type='ml.m4.2xlarge',
                                    
    train_use_spot_instances=True,                           # Use a spot instance
    train_max_run=300,                                       # Max training time
    train_max_wait=600,                                      # Max training time + spot waiting time
                                    
    debugger_hook_config=DebuggerHookConfig(                 # Save training tensors
        s3_output_path='s3://{}/{}/debug'.format(bucket, prefix), 
        collection_configs=[
            CollectionConfig(
                name="metrics",
                parameters={
                    "save_interval": save_interval
                }
            ),
            CollectionConfig(
                name="feature_importance",
                parameters={
                    "save_interval": save_interval
                }
            )
        ],
    ),
    
    rules=[
        Rule.sagemaker(                                      # Configure debugger rule
            rule_configs.class_imbalance(),                  
            rule_parameters={
                "collection_names": "metrics"
            },
        ),
    ]
)

### Setting hyper parameters
Each built-in algorithm has a set of hyperparameters. Here are the ones for XGBoost: 
https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html

That probably looks a little weird :) Let's stick to three simple parameters:
* Build a binary classifier: 'binary:logistic'.
* Use the 'Area Under Curve' metric, a good metric for classifiers.
* Train for 100 rounds, with early stopping if the metric hasn't improved in 10 rounds.

In [ ]:
# https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst

xgb.set_hyperparameters(
    objective='binary:logistic', 
    eval_metric='auc', 
    num_round=100,
    early_stopping_rounds=10
)

We're all set. Let's train!

In [ ]:
xgb.fit(s3_data)

Let's check the status of our debug job.

In [ ]:
xgb.latest_training_job.rule_job_summary()

Now, let's load the tensors saved during training, and plot them.

In [ ]:
import smdebug
from smdebug.trials import create_trial

s3_output_path = xgb.latest_job_debugger_artifacts_path()
trial = create_trial(s3_output_path)

Let's plot our metric over time.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import re

def get_data(trial, tname):
    """
    For the given tensor name, walks though all the iterations
    for which you have data and fetches the values.
    Returns the set of steps and the values.
    """
    tensor = trial.tensor(tname)
    steps = tensor.steps()
    vals = [tensor.value(s) for s in steps]
    return steps, vals

def plot_collection(trial, collection_name, regex='.*', figsize=(8, 6)):
    """
    Takes a `trial` and a collection name, and 
    plots all tensors that match the given regex.
    """
    fig, ax = plt.subplots(figsize=figsize)
    sns.despine()

    tensors = trial.collection(collection_name).tensor_names

    for tensor_name in sorted(tensors):
        if re.match(regex, tensor_name):
            steps, data = get_data(trial, tensor_name)
            ax.plot(steps, data, label=tensor_name)

    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    ax.set_xlabel('Iteration')

In [ ]:
plot_collection(trial, "metrics")

Now let's plot feature importance.

In [ ]:
def plot_feature_importance(trial, importance_type="weight"):
    SUPPORTED_IMPORTANCE_TYPES = ["weight", "gain", "cover", "total_gain", "total_cover"]
    if importance_type not in SUPPORTED_IMPORTANCE_TYPES:
        raise ValueError(f"{importance_type} is not one of the supported importance types.")
    plot_collection(
        trial,
        "feature_importance",
        regex=f"feature_importance/{importance_type}/.*")

In [ ]:
plot_feature_importance(trial)

Features 1 (job) and 5 (housing) should be the most important ones.

___
# PART 3 - Deploying our model

Now let's deploy our model to an HTTPS endpoint, and enable data capture. All it takes is one line of code.

In [ ]:
from sagemaker.model_monitor.data_capture_config import DataCaptureConfig
# https://sagemaker.readthedocs.io/en/stable/model_monitor.html

from time import strftime, gmtime
timestamp = strftime('%d-%H-%M-%S', gmtime())

capture_path = 's3://{}/{}/capture/'.format(bucket, prefix)

xgb_endpoint = xgb.deploy(
    
    endpoint_name = 'DEMO-xgboost-dm-{}'.format(timestamp),
    
    initial_instance_count = 1,                    # Infrastructure requirements
    instance_type = 'ml.m4.xlarge',

    data_capture_config = DataCaptureConfig(       
        enable_capture = True,                     # Capture data
        sampling_percentage = 100,                 
        #capture_options = [“REQUEST”, “RESPONSE”] # Default value
        destination_s3_uri = capture_path          # Save data here
    )
)

## Predicting with our model

First we'll need to determine how we pass data into and receive data from our endpoint. Our data is currently stored as NumPy arrays in memory of our notebook instance. To send it in an HTTP POST request, we'll serialize it as a CSV string and then decode the resulting CSV.

Let's predict the first 10 samples from the test set.

In [ ]:
sm = boto3.Session().client(service_name='runtime.sagemaker') 

test_samples = [line.rstrip('\n') for line in open('test.csv')]
test_samples = test_samples[:100] # We'll predict the first 100 samples

for sample in test_samples:
    sample = bytes(sample, 'utf-8')
    response = sm.invoke_endpoint(EndpointName=xgb_endpoint.endpoint, 
                                  ContentType='text/csv', 
                                  Body=sample)
    print(response['Body'].read())

For each sample, our binary classifier returns a probability between 0 and 1. Since we decided to maximize accuracy, the model sets a threshold of 0.5: anything lower is treated as a 0, anything higher as a 1. 

To dive a little deeper:  the threshold is baked in the metric that XGBoost uses. Here, we use the default 'eval_metric' for classification, i.e. 'error'. This metric has a default threshold of 0.5. If you look at the XGBoost doc (https://xgboost.readthedocs.io/en/latest/parameter.html), you'll see that it's possible to pass a different threshold, doing something like:
xgb.set_hyperparameters(objective='binary:logistic', num_round=100, eval_metric='error@0.2')

In [ ]:
%%sh -s "$capture_path"
echo $1
sleep 120
aws s3 ls --recursive $1
aws s3 cp --recursive $1 .

In [ ]:
%%sh 

# head <FILENAME>

## Use batch prediction
Some use cases either don't require or don't work well with HTTPS-based prediction. Imagine having to predict 100GB of bulk data every 24 hours: it wouldn't be efficient to do this with an endpoint.

SageMaker supports batch prediction. Let's apply it to the model we trained earlier: run the next 2 cells and wait for a bit. While this takes place, head out to the SageMaker web console and familiarize yourself with the "Batch transform jobs" section.

In [ ]:
transformer = xgb.transformer(instance_count=1, instance_type='ml.m4.xlarge')

# Reminder: test.csv must only contain features, not the target value
transformer.transform('s3://{}/{}/test/test.csv'.format(bucket, prefix), content_type='text/csv')

In [ ]:
transformer.wait()
print(transformer.output_path)

### Copy the output file and display the first 10 predictions
Predictions are written to S3. Let's use the AWS CLI to retrieve them and display the first 10 probabilities.

In [ ]:
!aws s3 cp $transformer.output_path/test.csv.out .
!ls -l test.csv.out
!head -10 test.csv.out

## Deleting the endpoint
Once that we're done predicting, we can delete the endpoint (and stop paying for it). You can re-deploy again by running the appropriate cell above. 

In [ ]:
sagemaker.Session().delete_endpoint(xgb_endpoint.endpoint)